#### World Population Prospects 2019
#### File POP/1-1: Total population (both sexes combined) by region, subregion and country, ####annually for 1950-2100 (thousands)
#### Constant-mortality variant, 2020 - 2100
#### POP/DB/WPP/Rev.2019/POP/F01-1
#### © August 2019 by United Nations, made available under a Creative Commons license CC BY 3.0 IGO: #### http://creativecommons.org/licenses/by/3.0/igo/
#### United Nations, Department of Economic and Social Affairs, Population Division (2019). World Population Prospects 2019, Online Edition. Rev. 1.

In [1]:
import pandas as pd
import requests
import os
import csv
from googletrans import Translator

In [2]:
pd.set_option("display.precision", 2)

### OMS https://covid19.who.int/WHO-COVID-19-global-data.csv

In [3]:
direccion_url = 'https://covid19.who.int/WHO-COVID-19-global-data.csv'

'''Uncomment para definir el Path'''

direccion_local = '/Users/Dr.LuisEvelioRestrepoGarcia/datosRecopilador/CovidMundo.csv'    # En mi Apple
'''Obtiene la base de datos actualizada y la guardar en disco local.'''

datos_req = requests.get(direccion_url)
try:
    datos_req.raise_for_status()
except Exception as exc:
    print('Problema en la descarga: %s' % (exc))
print(datos_req.status_code == requests.codes.ok)
playFile = open('CovidMundo.csv', 'wb')
for chunk in datos_req.iter_content(100000):
    playFile.write(chunk)
playFile.close()

covMun = pd.read_csv('CovidMundo.csv', usecols=['Date_reported', ' Country', 
       ' New_cases', ' Cumulative_cases', ' New_deaths', ' Cumulative_deaths'])

covMun.rename({'Date_reported': 'Fecha_reporte', ' Country': 'País', ' New_cases':'Total_Casos', 
                ' Cumulative_cases': 'Casos_acumulados', ' New_deaths':'Fallecidos', 
                ' Cumulative_deaths':'Muertes_acumuladas'}, axis='columns', inplace=True)
covMun = covMun.groupby('País')[['Total_Casos','Fallecidos']].sum()


covMun = covMun.reset_index()

covMun

True


,País,Total_Casos,Fallecidos
0,Afghanistan,0,0
1,Albania,0,0
2,Algeria,0,0
3,American Samoa,0,0
4,Andorra,0,0
...,...,...,...
230,Wallis and Futuna,0,0
231,Yemen,0,0
232,Zambia,0,0
233,Zimbabwe,0,0


In [4]:
translator = Translator()
covMun['Spanish'] = covMun['País'].apply(translator.translate, src='en', dest='es').apply(getattr, args=('text',))
covMun.to_csv('CovidMundoT.csv')
covMun.drop('País', axis= 'columns', inplace=True)
covMun.rename({'Spanish':'País'}, axis = 'columns', inplace=True)
covMun.set_index('País', inplace=True)

In [5]:
mundo= pd.read_csv('poblacionMundial.csv')
mundo.rename({'Spanish':'País'}, axis='columns', inplace=True)
mundo.set_index('País', inplace=True)
mundo = mundo['2020']

In [6]:
Tasas_mundiales = pd.merge(covMun, mundo, left_index=True, right_index=True)

Tasas_mundiales.rename({'2020':'Población'}, axis ='columns', inplace=True)

Tasas_mundiales['Tasa de mortalidad por millón de habitantes']= (Tasas_mundiales['Fallecidos']/Tasas_mundiales['Población'])*1000000
Tasas_mundiales['Tasa de letalidad por cien habitantes']= (Tasas_mundiales['Fallecidos']/Tasas_mundiales['Total_Casos'])*100

In [7]:
Tasas_mundiales

,Total_Casos,Fallecidos,Población,Tasa de mortalidad por millón de habitantes,Tasa de letalidad por cien habitantes
País,,,,,
Afganistán,0,0,38928000,0.0,NaN
Albania,0,0,2878000,0.0,NaN
Argelia,0,0,43851000,0.0,NaN
Samoa Americana,0,0,55000,0.0,NaN
Andorra,0,0,77000,0.0,NaN
...,...,...,...,...,...
Venezuela (República Bolivariana de),0,0,28436000,0.0,NaN
Vietnam,0,0,97339000,0.0,NaN
Yemen,0,0,29826000,0.0,NaN


In [8]:
pais = ['Colombia', 'Brasil', 'Argentina', 'Chile',
        'Estados Unidos de America', 'Canadá',
        'España', 'Francia', 'Italia', 
        'Alemania', 'Suecia']

In [9]:
Tasas_mundiales.loc[pais].sort_values('Tasa de mortalidad por millón de habitantes')

,Total_Casos,Fallecidos,Población,Tasa de mortalidad por millón de habitantes,Tasa de letalidad por cien habitantes
País,,,,,
Colombia,0,0,50883000,0.0,NaN
Brasil,0,0,212559000,0.0,NaN
Argentina,0,0,45196000,0.0,NaN
Chile,0,0,19116000,0.0,NaN
Estados Unidos de America,0,0,331003000,0.0,NaN
Canadá,0,0,37742000,0.0,NaN
España,0,0,46755000,0.0,NaN
Francia,0,0,65274000,0.0,NaN
Italia,0,0,60462000,0.0,NaN


In [10]:
Tasas_mundiales.to_csv('Tasas_mundiales.csv')